In [1]:
import glob
import pandas as pd
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
fns = glob.glob('calc/*/energies.csv')

In [3]:
def read_file(fn):
    molid = fn.split('/')[1].strip('0')
    df = pd.read_csv(fn)
    df['molid'] = int(molid)
    df = df[~np.isnan(df.reference_energy)] # no missing reference values
    df = df[~df.targets.str.contains('He')] # no He in sites
    df = df[df.total_energy != df.reference_energy] # no identity operation
    return df.copy()
dfs = [read_file(_) for _ in fns]

In [4]:
df = pd.concat(dfs).reset_index(drop=True)

In [5]:
df.groupby('molid').count()['targets'].reset_index().sort_values('targets').tail()

,molid,targets
525,60128,16
877,113927,17
793,101717,17
854,110138,18
905,118466,20


In [6]:
df.query('molid == 92').sort_values('reference_energy').rank()

,targets,total_energy,reference_energy,molid
6103,10.0,1.0,1.0,6.0
6101,8.0,2.0,2.0,6.0
6102,9.0,4.0,3.0,6.0
6100,7.0,3.0,4.0,6.0
6097,4.0,9.0,5.0,6.0
6099,6.0,6.0,6.0,6.0
6098,5.0,7.0,7.0,6.0
6104,11.0,5.0,8.0,6.0
6095,2.0,8.0,9.0,6.0
6094,1.0,10.0,10.0,6.0


In [7]:
def analyse_rank(targetrank):
    molids = []
    poss = []
    for name, group in df.groupby('molid'):
        try:
            poss.append(np.where(group.sort_values('reference_energy').rank(numeric_only=True, method='first')['total_energy'].values == targetrank)[0][0]+1)
            molids.append(name)
        except:
            pass
    return molids, poss
def rank_everything(df):
    dfs = []
    for name, group in df.groupby('molid'):
        s = group.sort_values('reference_energy').rank(numeric_only=True, method='first')
        s.columns = 'total_rank reference_rank molid'.split()
        s.molid = name
        dfs.append(s.copy())
    return pd.concat(dfs)
        

In [9]:
df_rank = rank_everything(df)
df.to_pickle('apdft.pkl')
df_rank.to_pickle('apdft-ranked.pkl')